In [2]:
import numpy
import nibabel as nib
import os
import matplotlib.pyplot as plt

# Label creation

This section will guide you through how to create new labels.
If you want, you can manually add a label in any desired location. For what you will only need a mask. </br>

This examples addresses a, as of June 2024, current limitation from Total Seg CT, that is that we cannot differentiate CSF, WM or GM from the Spinal Cord. </br>

Given that the labeling from anatomical image will have a specific value, when we add a new label we must add it with a value that is not included in the look up table. For example, for CT whole-body Total-Seg we must use a number higher than 117 to avoid conflicts. </br>


Given the nature of my example I used Spinal Cord Toolbox (SCT): https://spinalcordtoolbox.com/index.html by De Leener B, Levy S, Dupont SM, Fonov VS, Stikov N, Louis Collins D, Callot V, Cohen-Adad J. SCT: Spinal Cord Toolbox, an open-source software for processing spinal cord MRI data. Neuroimage 2017. </br> 
In order to add them we need to first register the CT image to the PAM50 space. For this we can use the Spinal Cord segmentation and you can manually create landmarks of spine C2 and C5. With this files ready you can automatically register to PAM50 space and procede with creating new labels. </br> 

**I.** The trick to add a new label to semgnetation is to first invert the values of the mask so that the ROI has value of 0 and everywhere else 1. </br> 
We then create a whole where we want the mask to be by multiplying it with our main labeled file.

In [ ]:
sct_maths -i new_label.nii.gz -o inv.nii.gz -sub 1
sct_maths -i inv.nii.gz -o inv.nii.gz -mul -1
sct_maths -i temp.nii.gz -o no_label.nii.gz -mul inv.nii.gz

**II.** We need to make sure that the new label will have a non-conflictng value depending on the range of the look-up table. </br>
*Note that we use temp as it is better to create a copy and edit the copy instead of the original.*

In [10]:
#Now the body will have zeroes where the label is going to go
# We need to set the value of the label to a number that 
# does not interfere with the look-up table
# For this example, any number higher than 117 would work 
sct_maths -i reslice_label.nii.gz -o new_label.nii.gz -mul 16
sct_maths -i new_label.nii.gz -o new_label.nii.gz -mul 16

We choose to multiply twice because there is a maximum value of 40 to multiply in SCT_maths. We can then create a new nifti file with value of 256 for the mask and 0 elsewhere. </br>
**III.** Then we just need to add the new label file to the file with a whole from the label we want to input.

In [ ]:
sct_maths -i no_label.nii.gz -o pro_labels.nii.gz -add new_label.nii.gz

## Important notes
- This procedure is not exclusive to SCT and can be perform with any tool of your preference!
- Once a new label is created, if you want to add more labels you will have to work on top of any previously made new volume with added labels, so creating a temporal copy to test first is recommended. </br>
- Also recommended, check the mask before multplication as I have deleted some files by multiplying inappropriate inversed masks. </br>
- In the "example" folder you will find an example of a succesfull implementation of 2 new masks to the CT_wb data that adds labels 289 and 256 to the whole body dataset.